In [1]:
# beginner introduction to tensorflow
import tensorflow as tf
import keras


In [2]:
# print tensor flow version
print("TensorFlow version ", tf.__version__)

TensorFlow version  2.13.0-rc0


In [4]:
# load data from mnist
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# normalize the data
x_train, x_test = x_train/255, x_test/255

In [10]:
# build a sequential machine learning model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28), name="flatten"),
    tf.keras.layers.Dense(128, activation="relu", name="dense"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10) # dont use softmax at the end of activation layer, 
                              # makes calculation of loss difficult.
])
model.build()
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101770 (397.54 KB)
Trainable params: 101770 (397.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
# make prediction with data
prediction = model(x_train[:1]).numpy()
prediction
# input of tensorflow must be a list of elements even if it is just a single item

array([[-0.4457718 , -0.05964347, -0.5374733 ,  0.07596178, -0.03127022,
        -0.19606507, -0.1625329 , -0.64230007,  0.6267793 , -0.14814305]],
      dtype=float32)

In [15]:
# get softmax values for prediction
tf.nn.softmax(prediction).numpy() # .numpy() operator turns tensor flow tensor into a numpy array

array([[0.07000668, 0.10299896, 0.06387252, 0.1179575 , 0.10596325,
        0.089864  , 0.09292842, 0.05751595, 0.20461738, 0.09427532]],
      dtype=float32)

In [19]:
# we would define a loss function of sparse cross entropy becuase our y values have not been one hot encoded
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# calculate the loss of the network thus far
loss = loss_fn(y_train[:1], prediction)
loss.numpy()

2.4094577

In [20]:
# compile model by specifying the loss, optimizer and metircs
model.compile(loss=loss_fn, 
              optimizer="adam", 
              metrics=["accuracy"])

In [21]:
# fit to train the model to the data
model.fit(x_train,
          y_train, 
          epochs=3)

Epoch 1/3
1875/1875 [==============================] - 2s 754us/step - loss: 0.2888 - accuracy: 0.9176
Epoch 2/3
1875/1875 [==============================] - 1s 699us/step - loss: 0.1414 - accuracy: 0.9574
Epoch 3/3
1875/1875 [==============================] - 1s 703us/step - loss: 0.1044 - accuracy: 0.9687


In [22]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 0s - loss: 0.0855 - accuracy: 0.9726 - 214ms/epoch - 684us/step


[0.08549690991640091, 0.972599983215332]

In [49]:
# turn trained model into porbability model
prob_model = tf.keras.models.Sequential([
    model,
    tf.keras.layers.Softmax()
], name="prob_model")
prob_model.build()
prob_model.summary()

Model: "prob_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (None, 10)                101770    
                                                                 
 softmax_9 (Softmax)         (None, 10)                0         
                                                                 
Total params: 101770 (397.54 KB)
Trainable params: 101770 (397.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [50]:
prob_model(x_test[:5])
# model now returning probabilities

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.8636548e-07, 3.2799608e-08, 5.5297401e-06, 2.2877506e-05,
        2.6210508e-11, 1.6885321e-06, 2.2343561e-11, 9.9996543e-01,
        8.7614893e-08, 4.2019610e-06],
       [2.5417081e-05, 4.7298003e-04, 9.9744594e-01, 1.7856327e-03,
        4.6400728e-10, 2.5336939e-04, 1.0717102e-05, 2.5635044e-13,
        6.0965231e-06, 1.8214587e-09],
       [1.1998703e-05, 9.8826319e-01, 2.4189749e-03, 1.8842715e-04,
        7.7688001e-04, 1.8721682e-04, 1.2796877e-03, 4.4215019e-03,
        2.3970862e-03, 5.5146153e-05],
       [9.9984396e-01, 1.1745184e-08, 3.7632388e-05, 5.0541001e-07,
        8.0546727e-08, 4.3452906e-06, 6.9135996e-05, 2.4972413e-05,
        3.1322823e-08, 1.9122084e-05],
       [4.0882438e-05, 2.3136726e-08, 1.2747733e-05, 1.8349393e-08,
        9.9659145e-01, 2.9889274e-05, 9.6644681e-06, 7.9705947e-05,
        4.0159298e-06, 3.2315899e-03]], dtype=float32)>

In [28]:
import pandas as pd

data = {'name': ['Sam', 'John', 'Tim', 'Tom', 'Singh', 'Song', 'Gold', 'Simp'],
 'age': [21, 34, 54, 76, 23, 10, 23, 100]}
df = pd.DataFrame(data)

categories = pd.cut([8], bins=[9, 20, 40, 70, 100], labels=[ 0, 1, 2, 3])[0]

print(categories)

nan


In [13]:
# bucketing continous values within a range
max_value = 10
min_value = 0

num_buckets = 2
bins = []
gap = (max_value - min_value)/num_buckets
start_point = min_value - 0.0001
bins.append(start_point)
for i in range(num_buckets-1):
    start_point += gap
    bins.append(start_point)
bins.append(max_value)

bins    

[-0.0001, 4.9999, 10]

In [20]:
def get_bukets(min_value, max_value, num_buckets=10, shift=1e-3):
    # defines the total gap from minimum to maximum
    gap = max_value - min_value
    # defines the interval for each step based on number of buckes
    interval = gap/num_buckets
    start_point = min_value - shift
    bukets = []
    bukets.append(start_point)
    for i in range(num_buckets -1):
        start_point += interval
        bukets.append(start_point)
    bukets.append(max_value)
    
    return bukets

get_bukets(1, 10, 4)

[0.999, 3.249, 5.4990000000000006, 7.7490000000000006, 10]

In [1]:
hey, hi = (1,  2, 3)

ValueError: too many values to unpack (expected 2)

In [1]:
from collections import deque

data = deque([2, 3, 4])
data

deque([2, 3, 4])

In [2]:
data.appendleft(1)
data

deque([1, 2, 3, 4])

In [3]:
data.append(5)
data

deque([1, 2, 3, 4, 5])

In [10]:
data.rotate(-3)
data

deque([1, 2, 3, 4, 5])

In [20]:
list(data)[-3+1:]

[4, 5]

In [21]:
import numpy as np
data = np.array([[1,2,3],[4,5,6]])

In [23]:
np.max(data, axis=1)

array([3, 6])

In [25]:
my_list = [1, 2, 3] + [4, 5]

In [29]:
my_list[1: 3]

[2, 3]